In [ ]:
import os
from loguru import logger
import glob
from IPython.display import Image, display

from ultralytics import YOLO
import torch
from pennylane import numpy as np

os.chdir("..")
logger.info(f"Current directory: {os.getcwd()}")
HOME = os.getcwd()

from src.YOLO.yolo_funcs import quantum_predict
from src.nn.models.quantum.QCNN import QuantumCircuitModel
from src.utils.reshape_data import ReshapeDATA
from src.utils.run_conf import CargaConf

# Importing the Hojas Dataset

In [ ]:
HOME = HOME + "/dataset"
%cd {HOME}

model = YOLO(f"{HOME}/best.pt")

In [ ]:
results = model.predict(
    source=HOME + "/HojasSimulador",
    save=False,
    project="classic_results",
    name="inferencia_test",
    exist_ok=True
)

# Validate Custom Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

In [ ]:
image_size = 16
n_classes = 2
n_wires = np.ceil(np.log2(3*image_size**2)).astype(int)
conf = CargaConf(image_size, n_classes, n_wires)
name = "2hojas_QCNN.pth"

In [ ]:
modelq = QuantumCircuitModel(n_wires=n_wires,
    embedding=conf["embedding"],
    circuit=conf["ansatz"],
    measurement=conf["measurement"],
    weight_shapes=conf["weight_shapes"],
    reshaper=conf["reshaper"]
) 

modelq.to(device)
modelq.load_state_dict(torch.load(name))
modelq.eval()

In [ ]:
quantum_predict(modelq, results, image_size, "Hojas")

In [ ]:
for image_path in glob.glob(f"{os.getcwd()}/quantum_results/_hojas_lejos/*.jpg")[:]:
    display(Image(filename=image_path, width=600))
    print(image_path)